---
title: Árboles de decisión
jupyter: python3
theme: Montpellier
author:  Máster en Ciencias de Datos e Ingeniería de Computadores
      Minería de Datos - Preprocesamiento y clasificación
date: Octubre 2024
toc-title: Tabla de Contenidos
toc: true
toc-depth: 1
# classoption: compress
execute:
  echo: true
output:
  beamer_presentation:
    slide_level: 1
    includes:
      in_header: ./simple.txt
format:
  html:
    code-fold: false
    code-summary: "Muestra código"
    fig-width: 5
    fig-height: 3
    fig-align: left
  beamer:
    fig-width: 4
    fig-height: 2
  revealjs:
    theme: dark
    fig-align: left
    fig-height: 3
    fig-cap-location: margin
    smaller: true
---

# Arboles de decisión

En este notebook vamos a repasar los principales métodos de árboles de decisión vistos en teoría. 

Se aplica una evaluación completamente *naif* de los clasificadores, sin ningún esquema de validación. Esta tarea se deja al estudiante para repasar los contenidos de la primera sesión de prácticas.

Vamos a usar el dataset del breastCancer, e Iris.

In [ ]:
#| code-fold: true
import sklearn
import numpy as np # debe ser una versión previa a la 1.24 para ID3. En Anaconda, por ejemplo, usar conda install numpy=1.23
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
#from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from sklearn.datasets import load_breast_cancer, load_iris, load_wine
from sklearn.datasets import load_iris, load_diabetes
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import LabelEncoder
from IPython.display import Image
import graphviz

#from matplotlib import style
#style.use('dark_background')

breastCancer = datasets.load_breast_cancer()
wine = datasets.load_wine()
diabetes = datasets.load_diabetes()
X_b = breastCancer.data
y_b = breastCancer.target
students = pd.read_csv("student-por.csv", sep=';')
X_s = students.drop("G3",axis=1)
y_s = students.G3

# Encode the values
# Se explicará en preprocesamiento
encoders = {}
for col in X_s.select_dtypes(include='object'):
    encoders[col] = LabelEncoder().fit(X_s[col])
    X_s[col] = encoders[col].transform(X_s[col])

iris = datasets.load_iris()
X_i = iris.data
y_i = iris.target

# ID3

ID3 no está oficialmente en SKlearn, pero sí en el repositorio Pip. 

Hay varios paquetes:

- [decision-tree-id3](https://pypi.org/project/decision-tree-id3/): Requiere numpy previa a la 1.24 para que funcione.
- [ID3Classifier](https://pypi.org/project/ID3classifier/):  Actualizado.

In [ ]:
!pip install ID3classifier

---

El ID3 instalado tiene la misma interfaz de fit & predict de sklearn, por lo que podemos usarla en los pipelines de sklearn.
Vamos a realizar un ejemplo de ajuste sencillo:

In [ ]:
from ID3classifier import ID3
# Create an instance of the ID3 classifier
id3 = ID3(max_depth=4, min_samples_split=2)
id3.fit(X_s, y_s)

y_pred = id3.predict(X_s)
# id3.print_tree() No funciona, no sé por qué, pero no es importante
print("Informe completo\n",classification_report(y_s, y_pred))

## Usando entropy para comparar

Vamos a utilizar el método DecisionTreeClassifier de sklearn con la medida **entropy** para realizar la partición de los datos y comparamos con el resultado del paquete ad-hoc.

In [ ]:
id3_sk = tree.DecisionTreeClassifier(max_depth=4, min_samples_split=2, criterion='entropy')
id3_sk.fit(X_s, y_s)

y_pred = id3_sk.predict(X_s)
print("Informe completo\n",classification_report(y_s, y_pred))

## Visualizando el árbol

In [ ]:
#| code-fold: True
dot_data = tree.export_graphviz(id3_sk, out_file=None, feature_names=X_s.columns, impurity=False) 

# dot_data = tree.export_graphviz(id3_sk, out_file=None, 
#                      feature_names=X_s.columns,  
#                      filled=True, rounded=True,  
#                      special_characters=True)

graph = graphviz.Source(dot_data)
graph.render("tree_students")
display(graph)

# C4.5

El DecisionTree de Sklearn permite usar la medida de impureza de C4.5, pero no exactamente su criterio de selección de atributos, gestión de valores perdidos, etc.

Vamos a utilizar una implementación no oficial más fiel al C4.5 original de Quinlan.

In [ ]:
from c45 import C45

c_45 = C45(attrNames=X_s.columns)

c_45.fit(X_s, y_s)

y_pred = c_45.predict(X_s)
print("Informe completo\n",classification_report(y_s, y_pred))

# CART

En este caso, sklearn sí tiene una implementación de CART fiel y podemos usarla directamente sin software externo.

In [ ]:
cart = tree.DecisionTreeClassifier(criterion='gini')
cart.fit(X_i, y_i)

y_pred = cart.predict(X_i)

print("Informe completo\n",classification_report(y_i, y_pred))

---

Los métodos que generan los ficheros .dot pueden visuzalizarse usando graphviz (hay que instalarlo en conda/pip, también desde la web y las interfaces al lenguaje python).
Vamos a dibujar el último árbol de decisión generado.

In [ ]:
#| code-fold: True
import graphviz 

# https://stackoverflow.com/questions/52566756/no-module-named-graphviz-in-jupyter-notebook si tienes problemas con graphviz

dot_data = tree.export_graphviz(cart, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris") #Buscar el fichero iris.pdf en la carpeta de trabajo
display(graph)

# Ejercicios Propuestos

1. Aplica en iris y en students la separación vista en análisis para ver los estudios.
2. Probarlo con max_depth de 5, 10, y sin especificar, y comparar la diferencia.
3. Visualizar los árboles del ejercicio anterior.